In [ ]:
#prepare the application id counts file

In [34]:
!mkdir /home/ubuntu/data/fulltextdatacp

In [ ]:
applids_filename = '/home/ubuntu/data/fulltextdatacp/patent_applicationids_all_years.csv'
!rm $applids_filename
for year in reversed(range(2005,2018)): 
    s3filename = 's3://ncw210data/FullTextData/patent_claim_data_' + str(year) + '.csv'
    textdatafilename = '/home/ubuntu/data/fulltextdatacp/patent_claim_data_' + str(year) + '.csv'
    
    !aws s3 cp $s3filename $textdatafilename
        
    df_data = pd.read_csv(textdatafilename,header=None, usecols=[0])
    df_data.to_csv(applids_filename,index=False, sep=',', mode='a', header=False)
    !rm $textdatafilename

In [45]:
applids_filename = '/home/ubuntu/data/fulltextdatacp/patent_applicationids_all_years.csv'
df_data = pd.read_csv(applids_filename,header=None, usecols=[0],names=['appl_doc_number'])
df_data_counts = df_data.groupby(['appl_doc_number']).size().reset_index(name='count')

In [47]:
applid_counts_filename = '/home/ubuntu/data/fulltextdatacp/patent_applicationid_counts_all_years.csv'
df_data_counts.to_csv(applid_counts_filename,index=False, sep=',')

In [50]:
#upload to S3
!aws s3 cp /home/ubuntu/data/fulltextdatacp/patent_applicationids_all_years.csv s3://ncw210data/FullTextData/patent_applicationids_all_years.csv
!aws s3 cp /home/ubuntu/data/fulltextdatacp/patent_applicationid_counts_all_years.csv s3://ncw210data/FullTextData/patent_applicationid_counts_all_years.csv    

upload: ../data/fulltextdatacp/patent_applicationids_all_years.csv to s3://ncw210data/FullTextData/patent_applicationids_all_years.csv
upload: ../data/fulltextdatacp/patent_applicationid_counts_all_years.csv to s3://ncw210data/FullTextData/patent_applicationid_counts_all_years.csv


In [2]:
# import pandas as pd
# df_ids = pd.read_csv('/home/ubuntu/data/fulltextdatacp/patent_applicationid_counts_all_years.csv')
# df_id_counts = df_ids.groupby(['count']).size().reset_index(name='count123')
#print(df_id_counts)

In [ ]:
#prepare the grant data file

In [83]:
#!mkdir /home/ubuntu/data/grantdatacp
#!aws s3 cp s3://ncw210data/PatentGrantData/ /home/ubuntu/data/grantdatacp/ --recursive --only-show-errors --exclude "*" --include "patent_grant_data_2*.csv"

In [159]:
!rm /home/ubuntu/data/grantdatacp/patent_grant_data_all_years.csv
!cat /home/ubuntu/data/grantdatacp/*.csv > /home/ubuntu/data/grantdatacp/patent_grant_data_all_years.csv
filename = '/home/ubuntu/data/grantdatacp/patent_grant_data_all_years.csv'
!sed  -i '1i appl_doc_number,appl_date,patent_number,patent_date,patent_doc_kind,pat_number_of_days,length_of_grant,references_count,number_of_claims' $filename

In [162]:
#drop the appl_date column, remove duplciates
df_data = pd.read_csv(filename)
df_data = df_data.drop('appl_date', 1)
df_data = df_data.drop_duplicates(subset=['appl_doc_number'], keep=False)

#df_data['patent_num_prefix'] = df_data['patent_number'].apply(lambda x: re.findall('\d*\D+',x)[0] if (x and x[0].isalpha()) else ('UTL' if x else '') )
#regex = re.compile('[^a-zA-Z]')
#df_data['patent_num_prefix'] = df_data['patent_number'].apply(lambda x: regex.sub('', x) if (x and x[0].isalpha()) else ('UTL' if x else x))
#df_data[['references_count','number_of_claims']] = df_data[['references_count','number_of_claims']].apply(pd.to_numeric)

df_data.to_csv(filename,index=False, sep=',')

In [164]:
#upload to S3
!aws s3 cp /home/ubuntu/data/grantdatacp/patent_grant_data_all_years.csv s3://ncw210data/FullTextData/patent_grant_data_all_years.csv

upload: ../data/grantdatacp/patent_grant_data_all_years.csv to s3://ncw210data/FullTextData/patent_grant_data_all_years.csv


In [ ]:
#Now merge these extra columns to small cluster data files

In [14]:
import numpy as np
def combine_with_smalldata(p,c,smalldatafilespath,df_applcountsdata,df_grantdata):
    smalldatafilename = smalldatafilespath + str(p) + '_' + str(c) + '.csv'
    outputfilename = '/home/ubuntu/data/smalldataoutput/small_cluster_data_' + str(p) + '_' + str(c) + '.csv'
    df_smalldata = pd.read_csv(smalldatafilename)
    #, usecols=['appl_doc_number','appl_country','appl_date','pub_doc_number','pub_date','number_of_days','invention_title','abstract','claim_text','claim_text_stemmed','top_cluster','pr_0','pr_1','pr_2','pr_3','pr_4','pr_5','pr_6','pr_7','pr_8','pr_9'])
    df_joined = df_smalldata.reset_index().merge(df_applcountsdata, how="left",left_on = 'appl_doc_number', right_on = 'appl_doc_number').set_index('index')
    df_joined_grant = df_joined.reset_index().merge(df_grantdata, how="left",left_on = 'appl_doc_number', right_on = 'appl_doc_number').set_index('index')
    
    df_joined_grant.to_csv(outputfilename,index=False, sep=',')


In [ ]:
#call the function for second level clustered files
import numpy as np
applcountsfilename = '/home/ubuntu/data/fulltextdatacp/patent_applicationid_counts_all_years.csv'
grantfilename = '/home/ubuntu/data/grantdatacp/patent_grant_data_all_years.csv'
df_applcountsdata = pd.read_csv(applcountsfilename)
df_grantdata = pd.read_csv(grantfilename)
smalldatafilespath = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_'

for p in range(0,10):
    for c in range(0,10):
        combine_with_smalldata(p,c,smalldatafilespath,df_applcountsdata,df_grantdata)

In [3]:
#upload to S3
!aws s3 cp /home/ubuntu/data/smalldataoutput/ s3://ncw210data/FullTextDataClusters/ --recursive --only-show-errors

In [15]:
!aws s3 cp /home/ubuntu/data/smalldataoutput/ s3://ncw210data/NewClusterFiles/smalldatafiles/ --recursive --only-show-errors